# Training TF net

In [ ]:
import sys
sys.path.insert(0, '/braintree/home/bashivan/dropbox/Codes/3D_fMRI_CNN/tf_pipeline/')
import os
from tf_trainer import Trainer
import logging 
from tf_model import TFModel
from tf_dataset import TFDataset
from tf_trainer import log_info_string
import numpy as np
import tensorflow as tf
import numpy as np
FLAGS = tf.app.flags.FLAGS
os.environ['CUDA_VISIBLE_DEVICES'] = '0,2'

FLAGS.initial_learning_rate = 0.0001
FLAGS.batch_size = 64
FLAGS.num_gpus = 1
FLAGS.num_epochs_per_decay = 3
FLAGS.num_time_steps = 64
FLAGS.data_dir = '/braintree/data2/active/users/bashivan/Data/fmri_conv'


if FLAGS.fold_to_run == -1:
    fold_to_run = range(FLAGS.num_folds)
else:
    fold_to_run = [FLAGS.fold_to_run]

# Load the dataset
fold_pairs = []

model = TFModel()
dataset = TFDataset(data_dir=FLAGS.data_dir)
tr = Trainer(model=model, dataset=dataset)
print("Loading data...")
tr.load_data(random=False)

sub_nums = tr.subjects
subs_in_fold = np.ceil(np.max(sub_nums) / float(10))
# n-fold cross validation
fold_results = []
for i in fold_to_run:
    '''
    for each kfold selects fold window to collect indices for test dataset and the rest becomes train
    '''
    test_ids = np.bitwise_and(sub_nums >= subs_in_fold * (i), sub_nums < subs_in_fold * (i + 1))
    train_ids = ~ test_ids
    fold_pairs.append((np.nonzero(train_ids)[0], np.nonzero(test_ids)[0]))

train_dir = FLAGS.train_dir
log_info_string('Start working on fold(s) {0}'.format(fold_to_run))
for fold_num, fold in enumerate([fold_pairs[i] for i in fold_to_run]):
    log_info_string('Beginning fold {0} out of {1}'.format(fold_num + 1, len(fold_pairs)))
    FLAGS.train_dir = os.path.join(train_dir, str(fold_num))

    print('Splitting the data...')
    tr.split_data(fold)
    print('Preprocessing data...')
    tr.preprocess_data()

    fold_results.append(tr.train(fold_num=fold_num))

# Aggregate results and save as a pickle
fold_results = pd.concat(fold_results)
fold_results.to_pickle(
'cnn_{0}_results_sgd_{1}_fold{2}.pkl'.format(FLAGS.model_type, FLAGS.initial_learning_rate, ''.join([str(i) for i in fold_to_run])))
    

Loading data...
Start working on fold(s) [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Beginning fold 1 out of 10
Splitting the data...
Preprocessing data...
****************************************
subset: train
data_dir: /braintree/data2/active/users/bashivan/Data/fmri_conv
num_epochs_per_decay: 3
num_gpus: 1
train_dir: /braintree/data2/active/users/bashivan/results/temp/0
batch_size: 64
model_type: lstm
num_checkpoints_tosave: 5
initial_learning_rate: 0.0001
num_examples: None
num_epochs: 10
checkpoint_dir: .
learning_rate_decay_factor: 0.1
fold_to_run: -1
num_folds: 10
seed: 0
eval_dir: /om/user/bashivan/temp
num_time_steps: 64
log_device_placement: False
****************************************
Using ADAM optimizer...
Using default loss (softmax-Xentropy)...
Instructions for updating:
Use tf.losses.softmax_cross_entropy instead. Note that the order of the logits and labels arguments has been changed.


Instructions for updating:
Use tf.losses.softmax_cross_entropy instead. Note that the order of the logits and labels arguments has been changed.


Instructions for updating:
Use tf.losses.compute_weighted_loss instead.


Instructions for updating:
Use tf.losses.compute_weighted_loss instead.


Instructions for updating:
Use tf.losses.add_loss instead.


...

Instructions for updating:
Use tf.losses.compute_weighted_loss instead.


Instructions for updating:
Use tf.losses.compute_weighted_loss instead.


Instructions for updating:
Use tf.losses.add_loss instead.


Instructions for updating:
Use tf.losses.add_loss instead.


2017-08-19 10:43:37.353115: step 0, loss = 0.70 (46.4 examples/sec; 1.380 sec/batch)
2017-08-19 10:44:20.000584: step 100, loss = 0.67 (162.7 examples/sec; 0.393 sec/batch)
2017-08-19 10:45:00.915741: step 200, loss = 0.69 (167.3 examples/sec; 0.382 sec/batch)
2017-08-19 10:45:42.170119: step 300, loss = 0.63 (162.6 examples/sec; 0.394 sec/batch)
Epoch 1 of 10 took 150.673s
  training loss:		0.679893
  training accuracy:		57.52 %
  validation loss:		0.563739
  validation accuracy:		81.15 %
Test results:
  test loss:			0.746181
  test accuracy:		52.43 %
2017-08-19 10:46:37.865831: step 400, loss = 0.68 (161.1 examples/sec; 0.397 sec/batch)
2017-08-19 10:47:19.061624: step 500, loss = 0.66 (161.5 examples/sec; 0.396 sec/batch)
2017-08-19 10:48:00.421526: step 600, loss = 0.64 (164.2 examples/sec; 0.390 sec/batch)
Epoch 2 of 10 took 148.063s
  training loss:		0.605517
  training accuracy:		72.44 %
  validation loss:		0.544289
  validation accuracy:		78.54 %
2017-08-19 10:48:48.670811: ste

In [6]:
FLAGS = tf.app.flags.FLAGS

fold_num = 0
fold = fold_pairs[0]

FLAGS.train_dir = os.path.join(FLAGS.train_dir, str(fold_num))

print('Splitting the data...')
tr.split_data(fold)
print('Preprocessing data...')
tr.preprocess_data()

tr.train(fold_num=fold_num)


****************************************
subset: train
data_dir: /braintree/data2/active/users/bashivan/Data/fmri_conv_orig
num_epochs_per_decay: 3
num_gpus: 1
train_dir: /braintree/data2/active/users/bashivan/results/temp/0/0
batch_size: 64
model_type: lstm
num_checkpoints_tosave: 5
initial_learning_rate: 0.001
num_examples: None
num_epochs: 10
checkpoint_dir: .
learning_rate_decay_factor: 0.1
fold_to_run: -1
num_folds: 10
seed: 0
eval_dir: /om/user/bashivan/temp
num_time_steps: 64
log_device_placement: False
****************************************
Using ADAM optimizer...
Using default loss (softmax-Xentropy)...
Instructions for updating:
Use tf.losses.softmax_cross_entropy instead. Note that the order of the logits and labels arguments has been changed.
Instructions for updating:
Use tf.losses.compute_weighted_loss instead.
Instructions for updating:
Use tf.losses.add_loss instead.
total_params: 185650
2017-08-19 09:55:45.806484: step 0, loss = 0.69 (43.8 examples/sec; 1.461 sec/bat

KeyboardInterrupt: 

# Plotting saved results

In [4]:
import cPickle
import numpy as np
from scipy.io import loadmat

data = cPickle.load(open('/braintree/home/bashivan/dropbox/Codes/3D_fMRI_CNN_copy/results/conv_2_1/cnn_lstm_results_adam_0.001_LSO_fold1.mat'))

data.keys()

ImportError: No module named _lib._version

In [1]:
import cPickle
import pandas as pd

results = cPickle.load(open('/braintree/home/bashivan/dropbox/Codes/3D_fMRI_CNN/cnn_lstm_results_adam_0.0001_10fold_.pkl'))

In [2]:
results

,test_acc,test_loss,training_acc,training_loss,valid_acc,valid_loss
0,0.553496,0.688185,0.522710,0.694362,0.387447,0.711750
1,0.607521,0.803726,0.709158,0.586928,0.626589,0.782541
2,0.607521,0.803726,0.919431,0.342473,0.606462,0.928678
3,0.627913,0.990931,0.984994,0.249511,0.627648,0.914467
4,0.627913,0.990931,0.987840,0.243746,0.596663,0.986135
